In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [73]:
from utils.data_handling import *
from utils.sensor_optimisation import *
from parameters import *
import GPy
import scipy.stats
from sklearn.covariance import *
from utils.data_assimilation import *
import time



import matplotlib.pyplot as plt
np.random.seed(101)



### Importing the Tracer Data to Select the Optimisation set S

In [4]:
parameters['field_name'] = "Tracer"
parameters


{'i_start': 0, 'i_end': 988, 'crop': None, 'field_name': 'Tracer'}

In [5]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

100%|██████████| 1/1 [00:00<00:00,  6.24it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


### Working subset of the data : set S

In [6]:
S_human = human_level_subset(buildingshape, loc_df, 2,1 )
print("The remaining number of points is : ", str(len(S_human)))

The remaining number of points is :  37847


In [7]:
S_working = working_subset(data_df, loc_df, nbins = (25,25,25), threshold_sum = 10**-2 )
print("The remaining number of points is : ", str(len(S_working)))

The remaining number of points is :  57725


In [8]:
S = np.intersect1d(S_human,S_working)
print("The remaining number of points is : ", str(len(S)))

The remaining number of points is :  23643


In [9]:
#A_list = set_to_onehot(S,data_df.shape[0])
#save_vtu(ref_vtu[0],'FinalSelection',A_list)

### Import the Field of interest for computing the optimisation : TracerBackground / Pressure

In [10]:
parameters['i_end'] = 988
parameters['field_name'] = "Tracer"
parameters


{'i_start': 0, 'i_end': 988, 'crop': None, 'field_name': 'Tracer'}

In [11]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

100%|██████████| 1/1 [00:00<00:00,  6.39it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


In [12]:
X = loc_df.values[:,:] #[I,:dim]
Z = data_df.values[:,:] #[:,t].reshape(-1,1) # [I,t].reshape(-1,1)
Z.shape

(100040, 989)

## Data Assimilation 

In [145]:
#indLoc = np.loadtxt('Optimal Points/localpoints_optimal_eps10-6.txt').astype(int)
indLoc = np.loadtxt('Optimal Points/localpoints_TSVD25').astype(int)
#indLoc = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
MSExb, MSExDA = data_assimilation(indLoc,Z)
MSExb - MSExDA

[38726 14276 91348  5338 40994 29626 65851 65734   851  2293]
L2 norm of the background error 1.1354456364425276
L2 norm of the error in DA solution 9.894538200895631e-17


1.1354456364425276

In [146]:
s = time.time()
A_ball = ball_set(indLoc, loc_df,radius=10)
indLoc_rand = randomise_set(A_ball)   
data_assimilation(indLoc_rand,Z) 
t = time.time() - s
t

0 38726
1 14276
2 91348
3 5338
4 40994
5 29626
6 65851
7 65734
8 851
9 2293
[2739, 34696, 67279, 44651, 58253, 8646, 26554, 35272, 6291, 30862]
L2 norm of the background error 0.44854066258726055
L2 norm of the error in DA solution 2.0498694302890195e-17


61.178593158721924

In [147]:
n_rand = 1000
DA_results_rand = dict()
#A_ball = ball_set(indLoc, loc_df,radius=5)
for n in tqdm.tqdm(range(n_rand)):
    DA_results_rand[n] = dict()
    s = time.time()
    DA_results_rand[n]['indLoc'] = randomise_set(A_ball)   
    DA_results_rand[n]['MSExb'], DA_results_rand[n]['MSExDA'] = data_assimilation(DA_results_rand[n]['indLoc'],Z)
    DA_results_rand[n]['time'] = time.time() - s


  0%|          | 0/1000 [00:00<?, ?it/s]

[9408, 16710, 59678, 44651, 10048, 15190, 85094, 51272, 6290, 19637]



  0%|          | 1/1000 [00:00<08:25,  1.98it/s]

L2 norm of the background error 0.6944705768189489
L2 norm of the error in DA solution 1.8167613694079742e-17
[2739, 62362, 14791, 47616, 10048, 76271, 9487, 95412, 2813, 6862]



  0%|          | 2/1000 [00:01<08:33,  1.94it/s]

...

L2 norm of the background error 1.292831648199937
L2 norm of the error in DA solution 1.3255429591986974e-16
[20822, 74749, 49965, 44651, 12495, 20598, 20151, 15875, 3078, 62706]



100%|█████████▉| 999/1000 [08:13<00:00,  2.11it/s]

L2 norm of the background error 0.6382510293937471
L2 norm of the error in DA solution 5.437741343543121e-18
[93429, 49656, 18377, 47616, 1466, 46732, 63324, 51465, 35095, 12321]



100%|██████████| 1000/1000 [08:13<00:00,  2.11it/s]

L2 norm of the background error 0.6011059918733198
L2 norm of the error in DA solution 1.0778129133201316e-16


In [148]:
MSExDA_mean = []
MSExDA_std = 0
MSExb_mean = []
MSExb_std = 0
MSE_xDA_b = []
MSE_xDA_b_mean = 0
MSE_xDA_b_std = 0

for i in DA_results_rand:
    MSExDA_mean.append(DA_results_rand[i]['MSExDA'])
    MSExb_mean.append(DA_results_rand[i]['MSExb'])
    MSE_xDA_b.append(DA_results_rand[i]['MSExb'] - DA_results_rand[i]['MSExDA'])

MSExDA_std = np.std(MSExDA_mean)
MSExDA_mean = np.mean(MSExDA_mean)

MSExb_std = np.std(MSExb_mean)
MSExb_mean = np.mean(MSExb_mean)

MSE_xDA_b_std = np.std(MSE_xDA_b)
MSE_xDA_b_mean = np.mean(MSE_xDA_b)


In [149]:
MSExDA_mean, MSExDA_std

(6.684357982092262e-17, 4.945878862675774e-17)

In [150]:
MSExb_mean, MSExb_std

(0.6725417891658496, 0.21479092536782593)

In [151]:
MSE_xDA_b_mean, MSE_xDA_b_std

(0.6725417891658496, 0.2147909253678259)

In [131]:
MSExDA_mean / MSExDA

0.7251727281675493